<h1 align="center">Bitcoin Encryption Standard: Electrum BIE1 ECIES (WP1305)</h1>
<h3 align="center">Benjamin Watts</h3>
<h3 align="center">Researcher</h3>
<br>



<br>The encryption protocol we follow is Electrum BIE1 ECIES.<br>
<br>Setup:<br><br>
Alice has a public/private keypair $P_{A}^{DH}=S_{A}^{DH}⋅G$ and Bob has a keypair $P_{B}^{DH}=S_{B}^{DH}⋅G$ We assume that Alice and Bob have exchanged public keys off-chain and now Alice wishes to send an encrypted message to Bob on-chain, for example within an OP_RETURN statement. Alice will sign the on-chain transaction with a sperate keypair $P_{A}^{Tx}=S_{A}^{Tx}⋅G$ 

Summary table:
<h5 align="center">1. Establishing a Shared Secret</h3> 

||<img width=400/>|
|:----------------------------------:|:------------------------------------------------------------:|
|Alice's ECDH Key                    | $P_{A}^{DH}$|
|Bob's ECDH Key                      | $P_{B}^{DH}$|
|ECDH Shared Secret Point            | $S_{AB} = P_{A}^{DH} ⋅ P_{B}^{DH} = P_{B}^{DH} ⋅ P_{A}^{DH}$|
|ECDH Shared Secret Data             | $S_{AB}$ **(compressed)**|
|Hash Function for Shared Secret Data| **SHA-512**|

<h5 align="center">2. Message Encryption and Authentication</h3> 

||<img width=480/>|
|:----------------------------------:|:------------------------------------------------------------:|
|Message Padding                     | **PKCS7**                                     |
|Encryption Algorithm                | **AES 128-BIT CBC mode**                                     |
|Initialization Vector (IV)          | **First 128-bits of** $H(S)$                                 |
|Encryption Key                      | **Second 128-bits of** $H(S)$                                |
|HMAC Key                            | **Last 256-bits of** $H(S)$                                 |
|Message Authentication              | **First 128-bits of HMAC-SHA256 using entire ciphertext**    |

<h5 align="center">3. Transaction Composition</h3> 

||<img width=480/>|
|:----------------------------------:|:------------------------------------------------------------:|
|Tx Signature Key                    | $P_{A}^{Tx}$                                                 |
|Encryption Data Packet              | $P_{A}^{DH}$ **, ciphertext, HMAC**                          |
|Protocol Flag                       | **BIE1**                                                     |
|OP_RETURN Payload                   | **(BIE1)(**$P_{A}^{DH}$ **, ciphertext, HMAC)**              |

This protocol adheres to the following principles:<br>
1. Transaction signing keys are separate from encryption (Diffie-Hellman) keys. Advantages:
>a) Increased privacy and security.<br>
>b) The OP_RETURN payload can be used for secure off-chain messaging as well.<br>
>c) The protocol can easily be adapted to use Diffie-Hellman keys that do no belong to the bitcoin secp256k1 conventions. For example, with keys that have 256-bit security<br><br>
2. A new encryption key is used for each plaintext. Advantages:
>a) Assuming (1), if an audit takes place, then one can hand over decryption keys that will give the auditor power to view one plaintext only.



## Libraries and Modules

In [215]:
import binascii 
import hashlib
import ecdsa
import codecs
import hmac
import base64

from Crypto.Cipher import AES
from pkcs7 import PKCS7Encoder

## Elliptic Curve Field Setup

All key-related operations for Bitcoin occur within the finite group of secp256k1.<br>
Secp256k1, http://www.oid-info.com/get/1.3.132.0.10<br>

In [216]:
p_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
r_ec = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
b_ec = 0x0000000000000000000000000000000000000000000000000000000000000007
a_ec = 0x0000000000000000000000000000000000000000000000000000000000000000
Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(p_ec, a_ec, b_ec)
generator_secp256k1 = ecdsa.ellipticcurve.Point(curve_secp256k1, Gx, Gy, r_ec)
oid_secp256k1 = (1, 3, 132, 0, 10)
SECP256k1 = ecdsa.curves.Curve("SECP256k1", curve_secp256k1, generator_secp256k1, oid_secp256k1)

## Functions

In [217]:
def get_point_pubkey(point_input):
    if (point_input.y() % 2) == 1:
        result = '03' + '%064x' % point_input.x()
    else:
        result = '02' + '%064x' % point_input.x()
    return binascii.hexlify(codecs.decode(result, 'hex'))

def point_double():
    print("do this")

def point_add():
    print("do that")

## Alice's Details and Message
In the examples of this document, Alice is sending a message to Bob. Her private key, $S_{A}^{DH}$, is defined as the variable `SA` in the code below.


In [218]:
SA = 54221739386215982513329628131540269007441981031353650007031898434875003141355
# = L1Ejc5dAigm5XrM3mNptMEsNnHzS7s51YxU7J61ewGshZTKkbmzJ

PA = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, \
                               26186473731199907689170338364739255189550659682973035734804284247663992809959, \
                               71894484974716313150658193896579683660742267101889515604922967520820712029133, 10)

CompPA = '0339e504d6492b082da96e11e8f039796b06cd4855c101e2492a6f10f3e056a9e7'

message = 'Hello world'

# Bob's Details
Bob is the reciever.

In [219]:
SB = 19604546966363504345948514434989414564433960545638122907271979546904571708973
# = KxfxrUXSMjJQcb3JgnaaA6MqsrKQ1nBSxvhuigdKRyFiEm6BZDgG

PB = ecdsa.ellipticcurve.Point(ecdsa.SECP256k1.curve, \
                               60650801378751831856369186448697049492352593086860468010779747572550299104457, \
                               34835621696986631909597048749467048062767037690835130200819150414373328549789, 10)

CompPB = '03861723172e8f50bf4e3b2b70582619a7260b79f040bcb2460ffd1d9053f534c9'

## Establishing Shared Secret
**Input: The private key** $S_{A}^{DH}$ **and public key $P_{B}^{DH}$.**<br>
**Output: Shared Secret Data.**<br><br>
**Method:**<br>
The public key of Bob is multiplied by the private key of Alice to get a new point on the elliptic curve $S_{AB}=S_{A}^{DH}⋅P_{B}^{DH}$. This point is a secret that is only known to Alice and Bob, and should not be shared. From this the following shared secret data is obtained:
<h5 align="center"> $S = $ compressed format of $S_{AB}$ (33-bytes)</h5>



In [220]:
S = '031cc3959f09765b99d8323496bd1b27126c0dcc6ca369ec66b6cdef324301614a'

## Establishing symmetric keys
**Input: Shared Secret Data _S_.**<br>
**Output: IV, AES Key, HMAC Key.**<br><br>
**Method:**<br>
The secret data S is hashed by SHA512. The hash is then split into two, 128-bit blocks and one 256-bit block. These are used to calculate:
<h5 align="center"> IV = first 128-bits of SHA512(S),</h5>
<h5 align="center"> AES Key = second 128-bits of SHA512(S),</h5>
<h5 align="center"> HMAC Key = final 256-bits of SHA512(S)</h5>

<br>**Additional Remarks:**<br>
- Take note that the SHA512 function takes a bytes-like object only as its parameter. This may be a mutable 'bytearray' or immutable 'bytes' object.

In [221]:
data = bytearray.fromhex(S) # Important data type. Bytes-like object required for hashing
#data = bytes.fromhex(S)

IV = hashlib.sha512(data).digest()[0:16]
KAES = hashlib.sha512(data).digest()[16:32]
KHMAC = hashlib.sha512(data).digest()[32:64]

#print("IV as bytes:    " + str(IV))
#print("KAES as bytes:  " + str(KAES))
#print("KHMAC as bytes: " + str(KHMAC))

print("IV as hex:      " + str(binascii.hexlify(IV)))
print("KAES as hex:    " + str(binascii.hexlify(KAES)))
print("KHMAC as hex:   " + str(binascii.hexlify(KHMAC)))

IV as hex:      b'4a76311614a90d81a7c4f8d038c91f5f'
KAES as hex:    b'a4375fefdde407b0728ca4d35475cd67'
KHMAC as hex:   b'94d936f43d8c2e22cc81b52f8c28bfd5ac5e74c45bf3cabfbf145fc0f0d92337'


## Message Encryption
**Input: Plaintext, IV, AES Key.**<br>
**Output: Ciphertext.**<br><br>
**Method:**<br>
The plaintext is converted to hex and padded using Public Key Cryptography Standards Number 7 (PKCS7). The padded message is then encrypted using AES 128-bit CBC mode algorithm with the AES Key and IV as defined above. A rough outline of the method is as follows: 

1. The plaintext is split into 128-bit blocks.
2. The first block is then XOR’d with the IV. This ensures that if the same plaintext is encrypted twice with the same AES Key then then a different ciphertext will be produced (assuming the IV is not the same).
3. The result is fed into the AES 128 algorithm.

<br>**Additional Remarks:**<br>
- Take note that the `aes.encrypt` function requires bytes only, not bytearray.


In [222]:
encoder = PKCS7Encoder() # PKCS7 Encoder for padding
messagebuf = encoder.encode(message) # Pad message
messagebuf = binascii.hexlify(messagebuf.encode()) # Encoded in hex
messagebuf = messagebuf.decode() # "b'" removed
print("messagebuf: " + messagebuf)
messagebuf = bytes.fromhex(messagebuf) # Turned into a bytes object

aes = AES.new(KAES, AES.MODE_CBC, IV) 
encryptedBytes = aes.encrypt(messagebuf)

ciphertext = binascii.hexlify(encryptedBytes)
print("\nciphertext: " + str(ciphertext))

messagebuf: 48656c6c6f20776f726c640505050505

ciphertext: b'37ae432655fe95d444834c88e31b9c0e'


## Creating Hmac
**Input: Ciphertext, HMAC Key.**<br>
**Output: HMAC.**<br><br>
**Method:**<br>
Let us denote the HMAC Key as $K_{HMAC}$ and the hash function SHA-256 as $H$.<br>
Define opad= 0x5c*16, ipad= 0x36*16 and $m$ as the concatenated string of the Protocol Flag + Sender Public Key + Ciphertext. Calculate: 
<h5 align="center">$HMAC = H((K_{HMAC}⨁opad) ∥ (K_{HMAC}⨁ipad) ∥ m)).$</h5>
<br>The data is concatentated in the following order: Protocol Flag, Public Key, Ciphertext, HMAC. See Entire package.

In [223]:
BIE1 = '42494531'
pubKey = CompPA
encbuf = BIE1 + pubKey + ciphertext.decode()

encbuf = bytes.fromhex(encbuf) # bytes object important
h = hmac.new(KHMAC, encbuf, hashlib.sha256).hexdigest()

encbuf = binascii.hexlify(encbuf)
encbuf = encbuf.decode()
encbuf = encbuf + h

print("\nEntire package as hex:\n" + encbuf)
x = codecs.decode(encbuf, "hex")
print("\nEntire package as base-64:\n" + str(base64.b64encode(x).decode()))


Entire package as hex:
424945310339e504d6492b082da96e11e8f039796b06cd4855c101e2492a6f10f3e056a9e737ae432655fe95d444834c88e31b9c0ee70d8ca6c80c8ad83f0794ba03f9818f35a73691ad80b48e37e98fd150a0b49e

Entire package as base-64:
QklFMQM55QTWSSsILaluEejwOXlrBs1IVcEB4kkqbxDz4Fap5zeuQyZV/pXURINMiOMbnA7nDYymyAyK2D8HlLoD+YGPNac2ka2AtI436Y/RUKC0ng==


## Decryption: Slicing Entire Package
**Input: Entire Package.**<br>
**Output: Protocol Flag, Senders Public Key, Ciphertext, HMAC.**<br><br>
**Method:**<br>
The entire package is sliced into its constituent parts.
 
Bytes 0 to 4 define the Protocol Flag. 

Bytes 4 to 37 define the sender’s public key. 

Since the ciphertext is of variable length, depending on the size of the secret message, the length of the entire package must be used to determine where the ciphertext and HMAC start and end. Define entire_package.length as the length of the entire package in bytes. 
Bytes 37 to entire_package.length – 32 define the Ciphertext. 

Bytes entire_package.length – 32 to entire_package.length define the HMAC. 

Take note: The pkcs7 library's inbuilt decoder.decode() function isn't used as I can't get it to work. I did, however, pilfer the pkcs7_unpad function from pastebin (https://pastebin.com/zi11rc8h).

In [224]:
# Decrypt with entire_package and receivers private key
entire_package = 'QklFMQM55QTWSSsILaluEejwOXlrBs1IVcEB4kkqbxDz4Fap5zeuQyZV/pXURINMiOMbnA7nDYymyAyK2D8HlLoD+YGPNac2ka2AtI436Y/RUKC0ng=='
privKey = 'KxfxrUXSMjJQcb3JgnaaA6MqsrKQ1nBSxvhuigdKRyFiEm6BZDgG'
entire_package = base64.b64decode(entire_package)
entire_package = binascii.hexlify(entire_package)
entire_package = entire_package.decode()
length = len(entire_package)
protocol_flag = entire_package[0:8] # Python not dealing in bytes, in chars. 
pubkey = entire_package[8:74]
ciphertext = entire_package[74:length-64]
HMAC = entire_package[length-64:]

print(protocol_flag)
print(pubkey)
print(ciphertext)
print(HMAC)

42494531
0339e504d6492b082da96e11e8f039796b06cd4855c101e2492a6f10f3e056a9e7
37ae432655fe95d444834c88e31b9c0e
e70d8ca6c80c8ad83f0794ba03f9818f35a73691ad80b48e37e98fd150a0b49e


## Decryption: Establishing Shared Secret Data
**Input: The private key** $S_{B}^{DH}$ **and public key $P_{A}^{DH}$.**<br>
**Output: Shared Secret Data.**<br><br>
**Method:**<br>
The public key of Alice, as obtained from the Entire Package, is multiplied by the private key of Bob to get a new point on the elliptic curve $S_{AB}=S_{B}^{DH}⋅P_{A}^{DH}$

In [225]:
S = '031cc3959f09765b99d8323496bd1b27126c0dcc6ca369ec66b6cdef324301614a'
data = bytearray.fromhex(S)

## Decryption: Establishing Symmetric Keys
**Input: Shared Secret Data _S_.**<br>
**Output: IV, AES Key, HMAC Key.**<br><br>
**Method:**<br>
The secret data S is hashed by SHA512. The hash is then split into two, 128-bit blocks and one 256-bit block. These are used to calculate:
<h5 align="center"> IV = first 128-bits of SHA512(S),</h5>
<h5 align="center"> AES Key = second 128-bits of SHA512(S),</h5>
<h5 align="center"> HMAC Key = final 256-bits of SHA512(S)</h5>


In [226]:
IV = hashlib.sha512(data).digest()[0:16]
KAES = hashlib.sha512(data).digest()[16:32]
KHMAC = hashlib.sha512(data).digest()[32:64]

print(IV)
print(KAES)
print(KHMAC)

b'Jv1\x16\x14\xa9\r\x81\xa7\xc4\xf8\xd08\xc9\x1f_'
b'\xa47_\xef\xdd\xe4\x07\xb0r\x8c\xa4\xd3Tu\xcdg'
b'\x94\xd96\xf4=\x8c."\xcc\x81\xb5/\x8c(\xbf\xd5\xac^t\xc4[\xf3\xca\xbf\xbf\x14_\xc0\xf0\xd9#7'


## Decryption: Authenticity Check
**Input: Protocol Flag, Sender Public Key, Ciphertext, HMAC Key, HMAC.**<br>
**Output: TRUE/FALSE.**<br><br>
**Method:**<br>
To check the authenticity of the package, the receiver creates their own HMAC using the Protocol Flag, Senders Public Key and Ciphertext obtained from the Entire Package, as well as the HMAC Key derived from the hash of the shared secret. The value of this calculated HMAC is compared to the value of the HMAC of the entire package, found at bytes entire_package.length – 32 to entire_package.length. If the comparison is true, the receiver knows that the entire package is authentic. 

In [228]:
# not done

## Decryption: Message Decryption
**Input: Ciphertext, IV, AES Key.**<br>
**Output: Plaintext.**<br><br>
**Method:**<br>
The IV and AES Key are used as parameters to the AES-128 CBC Mode Decrypt algorithm. The output’s padding must be removed using the PKCS7 Decode function. The result is the secret message.

In [227]:
decipher = AES.new(KAES, AES.MODE_CBC, IV) 
ciphertext = binascii.unhexlify(ciphertext)
decryptedBytes = decipher.decrypt(ciphertext)

def pkcs7_unpad(data, blocksize=16):
    if data:
        padlen = data[-1]
        if 0 < padlen < blocksize:
            if data.endswith(bytes([padlen]) * padlen):
                return data[:-padlen]

print(pkcs7_unpad(decryptedBytes, blocksize=16).decode())

Hello world


# Big thank you to everyone who helped build this document!
# Including, Liam, Allison, Alex, Jad and Owen.